In [ ]:
#Import your data and necessary libraries

import requests
import json
import re
from collections import Counter
import nltk
from nltk.corpus import stopwords
from nltk import pos_tag, word_tokenize

BASE_URL = "https://api.genius.com"
TOKEN = "azYFZ12Bv1Lbkaynqu8LXuAODHgstzWU13R069C-z43iuAyWSc6faui-oriAkirC"
HEADERS = {"Authorization": f"Bearer {TOKEN}"}

def search_artist(artist_name):
    url = BASE_URL + "/search"
    params = {"q": artist_name}
    response = requests.get(url, params=params, headers=HEADERS)
    return response.json()

def get_song_info(song_id):
    url = BASE_URL + f"/songs/{song_id}"
    response = requests.get(url, headers=HEADERS)
    return response.json()

def extract_lyrics_from_url(url):
    # You will need BeautifulSoup to scrape lyrics because Genius doesn't return them via API
    import requests
    from bs4 import BeautifulSoup

    page = requests.get(url)
    html = BeautifulSoup(page.text, "html.parser")
    lyrics = html.find("div", class_="lyrics") or html.find("div", {"data-lyrics-container": "true"})
    return lyrics.get_text(separator=" ").strip()


ModuleNotFoundError: No module named 'bs4'

In [11]:
#Identify nounds, verbs, and adjectives in the lyrics using NLTK
def get_pos_counts(text):
    words = word_tokenize(text.lower())
    tagged = pos_tag(words)

    nouns = [w for w, t in tagged if t.startswith("NN")]
    verbs = [w for w, t in tagged if t.startswith("VB")]
    adjs  = [w for w, t in tagged if t.startswith("JJ")]

    return {
        "nouns": Counter(nouns),
        "verbs": Counter(verbs),
        "adjectives": Counter(adjs)
    }

In [12]:
FILLER = ["like", "yeah", "uh", "um", "you know", "huh", "ayy", "yo"]

def count_filler(text):
    text_low = text.lower()
    count = 0

    for f in FILLER:
        # handles both single and multi-word fillers
        count += len(re.findall(r"\b" + re.escape(f) + r"\b", text_low))

    return count

In [13]:
song_data = {
    "artist": "",
    "genre": "",
    "title": "",
    "release_year": "",
    "lyrics": "",
    "city": "",
    "gender": ""
}

In [14]:
top_words = {
    "rap": {
        "verbs": [],
        "nouns": [],
        "adjectives": []
    },
    "country": {
        "verbs": [],
        "nouns": [],
        "adjectives": []
    }
}

In [20]:
#Pick 3 Genres:
GENRES = {
    "rap": ["Drake", "Kendrick Lamar", "Travis Scott"],
    "pop": ["Taylor Swift", "Ariana Grande", "Olivia Rodrigo"],
    "country": ["Luke Combs", "Morgan Wallen", "Kane Brown"]
}


In [21]:
#Collect Data for each genre and store in song_data dictionary
all_songs = []

for genre, artists in GENRES.items():
    for artist in artists:
        search_result = search_artist(artist)
        hits = search_result["response"]["hits"][:5]  # take top 5 songs
        for hit in hits:
            song_id = hit["result"]["id"]
            song_title = hit["result"]["title"]
            song_info = get_song_info(song_id)
            url = song_info["response"]["song"]["url"]
            lyrics = extract_lyrics_from_url(url)
            clean_text = clean_lyrics(lyrics)
            pos_counts = get_pos_counts(clean_text)
            swear_count = count_swears(lyrics)
            filler_count = count_fillers(lyrics)

            all_songs.append({
                "genre": genre,
                "artist": artist,
                "song_title": song_title,
                "lyrics": lyrics,
                "clean_lyrics": clean_text,
                "top_nouns": pos_counts["nouns"].most_common(10),
                "top_verbs": pos_counts["verbs"].most_common(10),
                "top_adjectives": pos_counts["adjectives"].most_common(10),
                "swear_count": swear_count,
                "filler_count": filler_count
            })

# Convert to DataFrame
df = pd.DataFrame(all_songs)
df.to_csv("genre_lyrics_dataset.csv", index=False)
print("Dataset collected and saved! Shape:", df.shape)

ModuleNotFoundError: No module named 'bs4'